In [3]:
import numpy as np
import pandas as pd
import glob, os
import matplotlib.pyplot as plt
import cv2
import dlib
import imutils
from imutils import face_utils
from sklearn.metrics import r2_score
from scipy.spatial import distance as dist
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
# utility functions

# calcultes ear
def eye_aspect_ratio(eye):
    
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    
    C = dist.euclidean(eye[0], eye[3])
    
    # eye aspect ratio
    ear = (A + B) / (2 * C)
    
    return ear

# calculates MAR
def mouth_aspect_ratio(l):
    
    A = dist.euclidean(l[50],l[60])
    B = dist.euclidean(l[51],l[59])
    C = dist.euclidean(l[52],l[58])
    D = dist.euclidean(l[53],l[57])
    E = dist.euclidean(l[54],l[56])
    F = dist.euclidean(l[49],l[55])
    
    mar = (A + B + C + D + E) / 2*F
    
    return mar
    

# return coordinates of bounding box for detected face (HOG + Linear SVM)
def rect_to_bb(rect):
    x1 = rect.left()
    y1 = rect.top()
    x2 = rect.right()
    y2 = rect.bottom()
    
    return (x1, y1, x2, y2)


# convert the landmarks object into numpy array
def landmarks_to_np(landmarks, dtype='int'):
    coords = np.zeros((68, 2), dtype=dtype)
    
    for i in range(0, 68):
        coords[i] = (landmarks.part(i).x, landmarks.part(i).y)
        
    # return numpy array of landmarks object
    return coords;

# reads pts file
def read_pts(filename):
    return np.loadtxt(filename, comments=("version:", "n_points:", "{", "}"))

# draws image after detecting landmarks
def draw_image(test_image):
    
    # convert image to grayscale
    gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
    
    # histogram equilization
    image = cv2.equalizeHist(gray)
    
    # detects face
    faces = detector(gray)
    
    for face in faces:
        # get bounding box coordinates for each face
        x1, y1, x2, y2 = rect_to_bb(face)

        # draws rectangle around the face
        cv2.rectangle(test_image, (x1, y1), (x2, y2), (0, 255, 0), 3)

        # returns all landmarks feature in face
        landmarks = predictor(gray, face)

        # convert landmarks object to numpy array
        all_points = landmarks_to_np(landmarks)
        predicted_points = all_points[37:49]
    #         cv2.imshow('image', test_image)
    #         cv2.waitKey(0)
    #         cv2.destroyAllWindows()
        return predicted_points 
    
# pre-process image (grayscale + histogram equalization)
def pre_process(frame):
    # convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # histogram equilization
    image = cv2.equalizeHist(gray)
    
    return image

In [3]:
# Importing models

# loading landmark predictor
model = './models/shape_predictor_68_face_landmarks.dat'

# HOG + Linear SVM
detector = dlib.get_frontal_face_detector()

# Ensemble of Regression Trees
predictor = dlib.shape_predictor(model)

# Loading face detector (MobileNet)
prototxtPath=os.path.sep.join([r'C:\Users\suchika\Desktop\Projects\Driver Monitoring System\Face Detection Model','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\suchika\Desktop\Projects\Driver Monitoring System\Face Detection Model','res10_300x300_ssd_iter_140000.caffemodel'])

# CNN based
faceNet=cv2.dnn.readNet(prototxtPath,weightsPath)
maskNet=load_model(r'C:\Users\suchika\Desktop\Projects\Driver Monitoring System\MaskTrain.model')

In [4]:
# Function for detection of faces and mask prediction
def detect_and_predict_mask(frame,faceNet,maskNet):
    #grab the dimensions of the frame and then construct a blob
    (h,w)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1.0,(300,300),(104.0,177.0,123.0))
    
    faceNet.setInput(blob)
    detections=faceNet.forward()
    
    #initialize our list of faces, their corresponding locations and list of predictions
    faces=[]
    locs=[]
    preds=[]
    
    for i in range(0,detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>0.5:
        #we need the X,Y coordinates
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype('int')
        
            #ensure the bounding boxes fall within the dimensions of the frame
            (startX,startY)=(max(0,startX),max(0,startY))
            (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
            #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
            face=frame[startY:endY, startX:endX]
            face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face=cv2.resize(face,(224,224))
            face=img_to_array(face)
            face=preprocess_input(face)
        
            faces.append(face)
            locs.append((startX,startY,endX,endY))
        
        #only make a predictions if atleast one face was detected
        if len(faces)>0:
            faces=np.array(faces,dtype='float32')
            preds=maskNet.predict(faces,batch_size=12)
        
        return (locs,preds)

## Testing on Video Stream

In [7]:
# state
EYE_AR_THRESHOLD = 0.3
EYE_AR_CONSEC_FRAMES = 3
DROWSY_CONSEC_FRAMES = 20

HEAD_TILT_CONSEC_FRAMES = 20

FRAME_COUNTER = 0
BLINK_FLAG_COUNTER = 0
DROWSY_FLAG_COUNTER = 0

NO_BLINKS = 0
LAST_BLINK = None
BLINK_RATE_INTERVAL = 5

In [8]:
model = './models/eye_eyebrows_predictor.dat'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(model)

lStart, lEnd = 10, 16
rStart, rEnd = 16, 22

In [5]:
video = cv2.VideoCapture(0)

while True:
    
    FRAME_COUNTER += 1
    # fetch check and frame
    check, frame = video.read()
    
    # pre-process image (grayscale + histogram equalization)
    image = pre_process(frame)
    
    # get faces from detector
    faces = detector(image)
    
    # faces for mask detection
    facesM = []
    
    # locs for mask detection
    locsM = []
    
    # iterate over each face
    for face in faces:   
        # get bounding box coordinates for each face
        x1, y1, x2, y2 = rect_to_bb(face)
        
        locsM.append((x1, y1, x2, y2))
        
        # pre-processing face for mask detection
        faceM=frame[y1:y2, x1:x2]
        faceM=cv2.cvtColor(faceM,cv2.COLOR_BGR2RGB)
        faceM=cv2.resize(faceM,(224,224))
        faceM=img_to_array(faceM)
        faceM=preprocess_input(faceM)
        
        facesM.append(faceM)
        
        # draws rectangle around the face
        # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

        # returns all landmarks feature in face
        landmarks = predictor(image, face)
        
        # convert landmarks object to numpy array
        landmarks = face_utils.shape_to_np(landmarks)

        # gets eyes from landmarks array (landmarks contains 22 points)
        leftEye = landmarks[lStart: lEnd]
        rightEye = landmarks[rStart: rEnd]
        
        x1, y1 = leftEye[3][0], leftEye[3][1]
        x2, y2 = rightEye[0][0], rightEye[0][1]
        slope = (y2-y1)/(x2-x1)
        cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0))

        # gets eyes from landmarks array (landmarks contains 68 points)
        leftEye = landmarks[lStart: lEnd]
        rightEye = landmarks[rStart: rEnd]
        
        # calculates ear for each eye
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        # calculates mar
        mar = mouth_aspect_ratio(landmarks)
        
        # calculate average ear
        ear = (leftEAR + rightEAR) / 2
        
        # gets convexHull for both eyes and mouth
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        
        # draw contours around the eye and mouth
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        if (LAST_BLINK is not None and ((time.time() - LAST_BLINK) > BLINK_RATE_INTERVAL)):
            cv2.putText(frame, "Gaze Detected!", (220, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
        # if head tilt
        if slope < -0.4 or slope > 0.4:
            cv2.putText(frame, "Keep your head straight!", (200, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # if eye is closed
        if ear < EYE_AR_THRESHOLD:
            BLINK_FLAG_COUNTER += 1
            DROWSY_FLAG_COUNTER += 1
            
            if (DROWSY_FLAG_COUNTER > DROWSY_CONSEC_FRAMES):
                LAST_BLINK = None
                cv2.putText(frame, "Drowsiness Detected!", (200, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
        else:
            # if eye was closed for more than EYE_AR_CONSEC_FRAMES value
            if BLINK_FLAG_COUNTER > EYE_AR_CONSEC_FRAMES:
                NO_BLINKS += 1
                LAST_BLINK = time.time()
                
            # reset FLAG_COUNTER
            BLINK_FLAG_COUNTER = 0
            DROWSY_FLAG_COUNTER = 0
            
        # prints blinks
        cv2.putText(frame, "Blinks: {}".format(NO_BLINKS), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
    
    fps = video.get(cv2.CAP_PROP_FPS)
    cv2.putText(frame, "Fps: {}".format(fps), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    if len(faces)>0:
        facesM=np.array(facesM,dtype='float32')
        preds=maskNet.predict(facesM,batch_size=12)

    for (box,pred) in zip(locsM,preds):
        (startX,startY,endX,endY)=box
        (mask,withoutMask)=pred

        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)

        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)

        cv2.putText(frame,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)

        cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
    
    # display frame
    cv2.imshow('Frame', frame)
    
    # gets key pressed
    key = cv2.waitKey(1)
    
    # breaks if key is q
    if key == ord('q'):
        break
        
video.release()
cv2.destroyAllWindows()


NameError: name 'FRAME_COUNTER' is not defined